<div style="font-size: 200%; font-weight: bold; color: maroon;">Procesado de la red de vuelos con GraphFrames - I</div>
<!-- Paulo Villegas, 2021 - v. 1.2 -->

_Nota: algunas operaciones sobre el grafo (por ejemplo el cálculo de componentes conectados) consumen una cantidad apreciable de memoria. Esto hace que con la memoria por defecto de la máquina virtual (2 GB) el tiempo de proceso sea significativo (algunos minutos). Si se dispone de ella, conviene ampliar la memoria asignada a la máquina virtual en el `Vagrantfile`_

In [ ]:
spark.conf.get('spark.master')

# 1. Leer datos

Los datos de aeropuertos y rutas están almacenados en ficheros CSV. Los cargamos en DataFrames de Spark

In [ ]:
airports = spark.read.csv('DATA/openflights/airports.csv.bz2', inferSchema=True, header=True, nullValue='\\N')

In [ ]:
routes = spark.read.csv('DATA/openflights/routes.csv.bz2', inferSchema=True, header=True,  nullValue='\\N')

Esquema de datos leídos

In [ ]:
airports.printSchema()

In [ ]:
routes.printSchema()

Tamaño de los datos

In [ ]:
airports.count(), routes.count()

Nivel de particionado

In [ ]:
routes.rdd.getNumPartitions(), airports.rdd.getNumPartitions()

Algunas consultas

In [ ]:
routes.groupBy('stops').count().toPandas()

In [ ]:
airports.filter(airports['IATA'] == 'MAD').toPandas()

In [ ]:
airports.filter(airports.Country == 'Namibia').toPandas()

# 2. Construcción del grafo

In [ ]:
import graphframes as gf

## 2.1 Adaptar nodos y enlaces

In [ ]:
vertices = airports.withColumnRenamed('AirportId', 'id')

In [ ]:
edges_full = routes.withColumnRenamed('SourceAirportId', 'src').withColumnRenamed('DestAirportId', 'dst')

In [ ]:
edges_full.count()

## 2.2 Filtrado de enlaces espúreos

Eliminamos las rutas cuyo aeropuerto de entrada o de salida no está en la lista de nodos

In [ ]:
airport_list = vertices.rdd.map(lambda r: r.id).collect()

In [ ]:
airport_list = set(airport_list)

In [ ]:
edges = edges_full.filter( edges_full['src'].isin(airport_list) & edges_full['dst'].isin(airport_list) )

In [ ]:
edges.count()

<div class="alert alert-warning" role="alert" style="margin-top: 24pt; margin-bottom: 0pt;">El resultado de esta operación es correcto, pero no se ha implementado de la forma adecuada.<br>
    Ejercicio: averigüe por qué y reescribala usando una forma más apropiada</div>

## 2.3 Creación del grafo

In [ ]:
g = gf.GraphFrame(vertices, edges)

# 3. Grabar a disco un grafo

In [ ]:
g.vertices.write.mode('overwrite').parquet('OUT/flights-vertices')
g.edges.write.mode('overwrite').parquet('OUT/flights-edges')

# 4. Operaciones

## 4.1 Cálculo de grados

In [ ]:
g.degrees.printSchema()

In [ ]:
grados = g.degrees

In [ ]:
# Agrupamos por el grado y contamos cuántos hay con cada número de grado
grados.groupBy('degree').count().orderBy('count', ascending=False).toPandas()

In [ ]:
# ORdenamos ahora por grado, de mayor a menor
grados.orderBy('degree', ascending=False ).limit(10).toPandas()

In [ ]:
# Añadimos los nombres de los aeropuertos, haciendo un join con el DataFrame de vértices
airport_names = vertices.select('id', 'Name', 'City')
ordered_degrees = grados.join(airport_names, 'id').orderBy('degree', ascending=False )

In [ ]:
ordered_degrees.limit(10).toPandas()

<div class="alert alert-warning" role="alert" style="margin-top: 24pt; margin-bottom: 0pt;">Nota: estos valores son mucho más altos que los obtenidos en NetworkX sobre la misma red.<br>Ejercicio: averiguar por qué y proponer una solución.</div>

## 4.2 Cálculo de componentes conectados

In [ ]:
%%time

sc.setCheckpointDir('tmp/cc')
cc = g.connectedComponents()

In [ ]:
cc.count()

In [ ]:
cc.printSchema()

Veamos el contenido de alguno de los componentes

In [ ]:
c2 = cc.filter(cc.component == '1998').sort('Country', 'City')

In [ ]:
c2.toPandas()

Calculemos el tamaño de cada componente generado
<div class="alert alert-warning" role="alert" style="margin-top: 24pt; margin-bottom: 0pt;">Ejercicio: escriba las líneas necesarias</div>

In [ ]:
###

## 4.3 Búsqueda de caminos

In [ ]:
import pandas as pd
pd.set_option('max_colwidth', 200)

In [ ]:
ruta1 = g.bfs("IATA = 'MAD'", "IATA = 'JFK'")

In [ ]:
ruta1.toPandas()

In [ ]:
ruta2 = g.bfs("IATA = 'MAD'", "Country = 'Mongolia'")

In [ ]:
ruta2.toPandas()

In [ ]:
ruta3 = g.bfs("City = 'Vigo'", "City = 'Mendoza'")

In [ ]:
ruta3.toPandas()

In [ ]:
vertices.filter(vertices.id == 2650).toPandas()

## 4.4 Búsqueda de patrones

El esquema típico es:
1. Selección de subgrafos que siguen un determinado patrón
2. Filtrado de los subgrafos por propiedades de los nodos o los enlaces

Por ejemplo: recuperemos subgrafos posibles de 3 enlaces (es decir, vuelos origen-destino con 2 escalas intermedias)

In [ ]:
motif1 = g.find("(a)-[e1]->(b); (b)-[e2]->(c); (c)-[e3]->(d)")

In [ ]:
motif1.printSchema()

In [ ]:
%%time

motif1.count()

In [ ]:
motif1.limit(2).toPandas()

Quedémonos con las que hagan los 3 vuelos con la misma compañía

In [ ]:
motif2 = motif1.filter( (motif1['e1'].Airline == motif1['e2'].Airline) & 
                        (motif1.e2.Airline == motif1.e3.Airline) )

In [ ]:
motif2.count()


<div class="alert alert-warning" role="alert" style="margin-top: 24pt; margin-bottom: 0pt;">Ejercicio: añada ahora una condición adicional: los tres aeropuertos deben estar en países distintos</div>

In [ ]:
motif3 = ## completar

In [ ]:
motif3.count()

In [ ]:
from IPython.display import display
import pandas as pd

d = motif3.limit(10).toPandas()
with pd.option_context('display.max_colwidth', 100):
    display(d)